In [83]:
import warnings
warnings.filterwarnings("ignore")

In [84]:
import numpy as np 
import pandas as pd 
import pickle
from IPython.display import display

In [85]:
dec_orders = pd.read_csv('orders_before_dec.csv', sep='|')
jan_orders = pd.read_csv('orders_before_jan.csv', sep='|')


submission_dec = pd.read_csv('submission_dec.csv', sep='|')
submission_jan = pd.read_csv('submission_jan.csv', sep='|')

In [86]:
# X_train is all orders from 1. of June to 31. of October. y_train will be November data
X_train = dec_orders[(dec_orders['date'] < '2020-11-01')]
display(X_train)

# X_test is all orders from 1. of December to 31. of December. y_test will be January data
X_test = jan_orders[(jan_orders['date'] >= '2020-12-01') & (jan_orders['date'] <= '2020-12-31')]
X_test = X_test.sort_values(by=['userID', 'itemID'], ascending=True)
display(X_test)

,date,userID,itemID,order
0,2020-06-01,38769,3477,1
1,2020-06-01,42535,30474,1
2,2020-06-01,42535,15833,1
3,2020-06-01,42535,20131,1
4,2020-06-01,42535,4325,1
...,...,...,...,...
636347,2020-10-31,29360,29159,1
636348,2020-10-31,23841,1970,1
636349,2020-10-31,23841,15922,1
636350,2020-10-31,29360,12439,1


,date,userID,itemID,order
819667,2020-12-11,0,6446,1
819668,2020-12-11,0,20664,1
819666,2020-12-11,0,28231,1
779709,2020-12-04,0,31683,1
779710,2020-12-04,0,31923,1
...,...,...,...,...
832090,2020-12-13,46134,29345,1
797458,2020-12-07,46135,32204,3
893165,2020-12-28,46136,1782,5
776830,2020-12-02,46136,23669,1


In [87]:
# y_train will be December data
# make it so that we don't have to predict for (userID, itemID) combinations that has not appeared yet in training data
submission_nov = pd.merge(X_train, submission_dec, on=['userID', 'itemID'], how='inner')
submission_nov.drop_duplicates(subset=['userID', 'itemID'], keep='first', inplace = True) 
display(submission_nov)

,date,userID,itemID,order,prediction
0,2020-06-01,31089,23582,1,0
3,2020-06-01,2635,31850,3,0
5,2020-06-01,16922,31265,3,0
11,2020-06-01,10840,25358,2,0
15,2020-06-01,40278,18881,1,0
...,...,...,...,...,...
19434,2020-10-31,34711,7997,2,0
19435,2020-10-31,40953,29137,1,0
19436,2020-10-31,8746,10849,1,0
19437,2020-10-31,14256,23582,2,0


In [88]:
# make submission_nov like the original file format
submission_nov.drop(['order', 'date'], axis = 1, inplace = True)
submission_nov = submission_nov.sort_values(by=['userID', 'itemID'], ascending=True)
display(submission_nov)

,userID,itemID,prediction
1858,0,20664,0
2098,13,2690,0
17924,15,1299,0
19012,15,20968,0
9076,20,8272,0
...,...,...,...
10862,46118,20106,0
15503,46124,19677,0
1563,46125,12878,0
10957,46127,7963,0


In [89]:
# Write it to a csv file if you want
# submission_nov.to_csv('submission_nov.csv',sep='|',index=False)

In [90]:
# All examples from November 1-28
# can also use dec_orders bellow
nov_ordered = jan_orders[(jan_orders['date'] >= '2020-11-01') & (jan_orders['date'] <= '2020-11-28')]

# Right join the actual orders happening in november and "possible transactions" in november
nov_joined = pd.merge(nov_ordered, submission_nov, on=['userID', 'itemID'], how='right')

# Fill in correct predictions/goldstandard
nov_joined['prediction'] = 0

nov_joined['prediction'] = np.where((nov_joined['date'] >= '2020-11-01') & (nov_joined['date'] <= '2020-11-07'),1 ,nov_joined.prediction)
nov_joined['prediction'] = np.where((nov_joined['date'] >= '2020-11-08') & (nov_joined['date'] <= '2020-11-14'),2 ,nov_joined.prediction)
nov_joined['prediction'] = np.where((nov_joined['date'] >= '2020-11-15') & (nov_joined['date'] <= '2020-11-21'),3 ,nov_joined.prediction)
nov_joined['prediction'] = np.where((nov_joined['date'] >= '2020-11-22') & (nov_joined['date'] <= '2020-11-28'),4 ,nov_joined.prediction)

nov_joined.drop(['order', 'date'], axis = 1, inplace = True) 

# Take first week if person buyes same item multiple times in our test period
nov_joined.drop_duplicates(subset=['userID', 'itemID'], keep='first', inplace = True) 

y_train = nov_joined
display(y_train)

# Our data looks to be a bitt unbalanced with far more cases of not buying than buying in week 1, 2, 3 or 4.
tmp = y_train.prediction.value_counts()
tmp

,userID,itemID,prediction
0,0,20664,0
1,13,2690,0
2,15,1299,0
3,15,20968,0
4,20,8272,0
...,...,...,...
9514,46118,20106,2
9515,46124,19677,3
9516,46125,12878,3
9517,46127,7963,0


0    6205
2     909
4     887
3     750
1     648
Name: prediction, dtype: int64

In [91]:
# Create train test splitt
# Merge datasets, order and the splitt again
df = pd.merge(X_train, y_train, on=['userID', 'itemID'], how='left')
df['prediction'] = df['prediction'].fillna(0)
df = df.sort_values(by=['userID', 'itemID'], ascending=True)

display(df)

# Our data looks to be a bitt unbalanced with far more cases of not buying than buying
# kan ha noe med hvordan du lagde settet. Har jo bare med 1 2 3 4 verdier for 1 månde her.
tmp2 = df.prediction.value_counts()
tmp2

,date,userID,itemID,order,prediction
390966,2020-09-01,0,1505,1,0.0
268197,2020-08-03,0,12468,1,0.0
334333,2020-08-18,0,12505,1,0.0
268196,2020-08-03,0,15083,1,0.0
22146,2020-06-05,0,20664,1,0.0
...,...,...,...,...,...
453489,2020-09-17,46137,2667,1,0.0
287155,2020-08-08,46137,20209,1,0.0
287156,2020-08-08,46137,28343,1,0.0
287097,2020-08-08,46137,28900,2,0.0


0.0    629951
2.0      1834
4.0      1784
3.0      1470
1.0      1313
Name: prediction, dtype: int64

In [92]:
y_train = df[['prediction']]
display(y_train)

df.drop(['prediction'], axis = 1, inplace = True)
X_train = df
display(X_train)

,prediction
390966,0.0
268197,0.0
334333,0.0
268196,0.0
22146,0.0
...,...
453489,0.0
287155,0.0
287156,0.0
287097,0.0


,date,userID,itemID,order
390966,2020-09-01,0,1505,1
268197,2020-08-03,0,12468,1
334333,2020-08-18,0,12505,1
268196,2020-08-03,0,15083,1
22146,2020-06-05,0,20664,1
...,...,...,...,...
453489,2020-09-17,46137,2667,1
287155,2020-08-08,46137,20209,1
287156,2020-08-08,46137,28343,1
287097,2020-08-08,46137,28900,2


In [93]:
# X_1 = pd.DataFrame(data=pd.get_dummies(X.index.month, drop_first=True, prefix="month"))

def convert_dates(x):
    x['date']=pd.to_datetime(x['date'])
#     x['year']=x['date'].dt.year  #Change to day of the week? Year has no importance in our case
    x['month']=x['date'].dt.month
    x['day'] = x['date'].dt.day
    x['dayofweek']=x['date'].dt.dayofweek
    x.pop('date')
    return x

X_train = convert_dates(X_train)
X_test = convert_dates(X_test)
display(X_test)

,userID,itemID,order,month,day,dayofweek
819667,0,6446,1,12,11,4
819668,0,20664,1,12,11,4
819666,0,28231,1,12,11,4
779709,0,31683,1,12,4,4
779710,0,31923,1,12,4,4
...,...,...,...,...,...,...
832090,46134,29345,1,12,13,6
797458,46135,32204,3,12,7,0
893165,46136,1782,5,12,28,0
776830,46136,23669,1,12,2,2


In [94]:
# Join X_test and X_train on itemID for items.csv
items = pd.read_csv('items.csv', sep='|')

X_train = pd.merge(X_train, items, on=['itemID'], how='left')
X_train.drop(['categories'], axis = 1, inplace = True)
display(X_train)

X_test = pd.merge(X_test, items, on=['itemID'], how='left')
X_test.drop(['categories'], axis = 1, inplace = True)
display(X_test)

,userID,itemID,order,month,day,dayofweek,brand,feature_1,feature_2,feature_3,feature_4,feature_5
0,0,1505,1,9,1,1,286,4,0,82,0,144
1,0,12468,1,8,3,0,1288,10,0,421,3,3
2,0,12505,1,8,18,1,53,4,3,474,0,-1
3,0,15083,1,8,3,0,286,4,0,82,0,144
4,0,20664,1,6,5,4,408,4,0,284,0,66
...,...,...,...,...,...,...,...,...,...,...,...,...
636347,46137,2667,1,9,17,3,1301,4,0,148,0,144
636348,46137,20209,1,8,8,5,703,4,0,291,0,44
636349,46137,28343,1,8,8,5,842,10,1,503,0,137
636350,46137,28900,2,8,8,5,1156,10,0,137,3,87


,userID,itemID,order,month,day,dayofweek,brand,feature_1,feature_2,feature_3,feature_4,feature_5
0,0,6446,1,12,11,4,839,4,0,18,0,81
1,0,20664,1,12,11,4,408,4,0,284,0,66
2,0,28231,1,12,11,4,193,4,3,468,3,108
3,0,31683,1,12,4,4,1371,4,0,484,3,175
4,0,31923,1,12,4,4,1111,4,0,487,0,144
...,...,...,...,...,...,...,...,...,...,...,...,...
138745,46134,29345,1,12,13,6,393,4,0,275,3,151
138746,46135,32204,3,12,7,0,1496,10,0,345,3,95
138747,46136,1782,5,12,28,0,6,4,3,321,0,144
138748,46136,23669,1,12,2,2,703,4,0,291,0,44


In [30]:
# Decision trees frequently perform well on imbalanced data
# Weighted Random Forest with SMOTHE
# %%time
from sklearn.ensemble import RandomForestClassifier

Not easy for time series data (not any significant gain as far as i could see)
from imblearn.over_sampling import SMOTE

smote = SMOTE()

X_smote, y_smote = smote.fit_resample(X_train, y_train)

# Set class_weight='balanced' or =‘balanced_subsample‘.
# Try and use own optimization/eval method? (Not implemented)
clf = RandomForestClassifier(n_estimators=20, random_state=42, class_weight='balanced_subsample')
clf.fit(X_smote, y_smote)
print(clf)

filename = 'SmotheRandomForestClassifier_model.sav'
pickle.dump(clf, open(filename, 'wb'))

RandomForestClassifier(class_weight='balanced_subsample', n_estimators=20,
                       random_state=42)


In [31]:
loaded_model = pickle.load(open(filename, 'rb'))

In [32]:
pred = loaded_model.predict(X_test)  

In [13]:
from imblearn.ensemble import BalancedRandomForestClassifier

# class_weight='balanced_subsample'

model = BalancedRandomForestClassifier(random_state=42) 
model.fit(X_train, y_train)

filename2 = 'BalancedRandomForestClassifier_model.sav'
pickle.dump(model, open(filename2, 'wb'))

In [14]:
loaded_model2 = pickle.load(open(filename2, 'rb'))

In [15]:
pred = loaded_model2.predict(X_test)  

Maybe try penalize Algorithms (Cost-Sensitive Training)
Use penalized learning algorithms (SVM) that increase the cost of classification mistakes on the minority class.
Or weight towards balanced.
Takes to long

In [ ]:
#Try svm 
# from sklearn.svm import SVC

# svc_model = SVC(class_weight='balanced', probability=False)

# svc_model.fit(X_train, y_train)

# filename3 = 'SVM_model.sav'
# pickle.dump(model, open(filename3, 'wb'))

In [ ]:
# loaded_model3 = pickle.load(open(filename3, 'rb'))

In [ ]:
# pred = loaded_model3.predict(X_test)

In [64]:
# Try KNN (Best for now)
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

pred = neigh.predict(X_test)

In [95]:
# Try logistic regression: does not work in this case. 
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X_train, y_train)

lr = LogisticRegression(random_state=42).fit(X_smote, y_smote)
pred = lr.predict(X_test)

In [96]:
X_test['prediction'] = pred
display(X_test)

tmp4 = X_test.prediction.value_counts()
tmp4

,userID,itemID,order,month,day,dayofweek,brand,feature_1,feature_2,feature_3,feature_4,feature_5,prediction
0,0,6446,1,12,11,4,839,4,0,18,0,81,3.0
1,0,20664,1,12,11,4,408,4,0,284,0,66,4.0
2,0,28231,1,12,11,4,193,4,3,468,3,108,4.0
3,0,31683,1,12,4,4,1371,4,0,484,3,175,4.0
4,0,31923,1,12,4,4,1111,4,0,487,0,144,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
138745,46134,29345,1,12,13,6,393,4,0,275,3,151,2.0
138746,46135,32204,3,12,7,0,1496,10,0,345,3,95,2.0
138747,46136,1782,5,12,28,0,6,4,3,321,0,144,2.0
138748,46136,23669,1,12,2,2,703,4,0,291,0,44,2.0


3.0    41006
4.0    35767
2.0    26737
0.0    18172
1.0    17068
Name: prediction, dtype: int64

In [97]:
# Make/addapt to submission file
submission_jan.drop(['prediction'], axis = 1, inplace = True) 
submission_jan = pd.merge(X_test, submission_jan, on=['userID', 'itemID'], how='right')

#Fill inn 0 for the (user, items) combinations we have not seen before
submission_jan['prediction'] = submission_jan['prediction'].fillna(0) 
submission_jan.drop_duplicates(subset=['userID', 'itemID'], keep='first', inplace = True)
submission_jan.drop(['order', 'dayofweek', 'month', 'day', 'brand', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5'], axis = 1, inplace = True)
display(submission_jan)

tmp5 = submission_jan.prediction.value_counts()
tmp5

,userID,itemID,prediction
0,0,20664,4.0
1,0,28231,4.0
2,13,2690,3.0
3,15,1299,3.0
4,15,20968,3.0
...,...,...,...
10186,46118,20106,2.0
10187,46124,19677,2.0
10188,46125,12878,0.0
10189,46127,7963,3.0


0.0    6438
3.0    1176
4.0    1054
2.0     774
1.0     498
Name: prediction, dtype: int64

In [98]:
# Make submission file for january
submission_jan.to_csv('submission_jan.csv', sep='|', index=False)

In [ ]:
# Make a submission file for december, with only 0 as the predicted value
submission_dec['prediction'] = 0

In [ ]:
submission_dec.to_csv('submission_dec.csv', sep='|', index=False)